In [1]:
import asyncio
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.base import TaskResult

import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

# Basic Task Assignment and Assistant Agent

In [2]:
base_completion_client = OpenAIChatCompletionClient(
        model="qwen2.5",
        api_key="NotRequiredSinceWeAreLocal",
        base_url='http://host.docker.internal:11434/v1',
        model_capabilities={
            "json_output": False,
            "vision": False,
            "function_calling": True,
        },
        parallel_tool_calls=True
 )

In [3]:
# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


In [4]:
base_agent = AssistantAgent("assistant", 
                            base_completion_client,
                           tools=[web_search])

In [5]:
async def main() -> None:
    print("TASK 1\n")
    print(await base_agent.run(task="Say 'Hello World!'"))
    
    print("\n\nTASK 2\n")
    print(await base_agent.run(task="Search the web for Autogen"))

In [6]:
await main()

TASK 1

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="Say 'Hello World!'", type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=166, completion_tokens=22), content=[FunctionCall(id='call_peykc9r6', arguments='{"message":"Hello World!"}', name='echo_message')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content="Error: The tool 'echo_message' is not available.", call_id='call_peykc9r6')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_usage=None, content="Error: The tool 'echo_message' is not available.", type='ToolCallSummaryMessage')], stop_reason=None)


TASK 2

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Search the web for Autogen', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=227, completion_tokens=21)

In [7]:
#It is important to note that on_messages() will update the internal state of the agent 
# – it will add the messages to the agent’s history. So you should call this method with new messages. You should not repeatedly call this method with the same messages or the complete history.

async def assistant_run() -> None:
    response = await base_agent.on_messages(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print("\n-----\n")
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=286, completion_tokens=22), content=[FunctionCall(id='call_3gbikf6l', arguments='{"query":"information on AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_3gbikf6l')], type='ToolCallExecutionEvent')]

-----

source='assistant' models_usage=None content='AutoGen is a programming framework for building multi-agent applications.' type='ToolCallSummaryMessage'


# Streaming

In [8]:
async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream (as shown in the previous example).
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Find information on AutoGen", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        base_agent.on_messages_stream(
            [TextMessage(content="Find information on AutoGen", source="user")],
            cancellation_token=CancellationToken(),
        )
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

---------- assistant ----------
[FunctionCall(id='call_93wtikku', arguments='{"query":"AutoGen documentation and examples"}', name='web_search')]
---------- assistant ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_93wtikku')]
---------- assistant ----------
AutoGen is a programming framework for building multi-agent applications.


# Use Langchain Tools

In [9]:
df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"df": df}))
agent = AssistantAgent(
    "assistant", tools=[tool], model_client=base_completion_client, system_message="Use the `df` variable to access the dataset."
)
await Console(
    agent.on_messages_stream(
        [TextMessage(content="What's the average age of the passengers?", source="user")], CancellationToken()
    )
)


---------- assistant ----------
[FunctionCall(id='call_4u9w3amb', arguments='{"query":"df[\'age\'].mean()"}', name='python_repl_ast')]
---------- assistant ----------
[FunctionExecutionResult(content="KeyError: 'age'", call_id='call_4u9w3amb')]
---------- assistant ----------
KeyError: 'age'


Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content="KeyError: 'age'", type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=199, completion_tokens=27), content=[FunctionCall(id='call_4u9w3amb', arguments='{"query":"df[\'age\'].mean()"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content="KeyError: 'age'", call_id='call_4u9w3amb')], type='ToolCallExecutionEvent')])

## RoundRobinGroupChat with Agent Team

In [41]:
# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."


async def main() -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=base_completion_client,
        tools=[get_weather],
    )

    # Define a team with a single agent and maximum auto-gen turns of 1.
    agent_team = RoundRobinGroupChat([weather_agent], max_turns=1)

    while True:
        # Get user input from the console.
        user_input = input("Enter a message (type 'exit' to leave): ")
        if user_input.strip().lower() == "exit":
            break
        # Run the team and stream messages to the console.
        stream = agent_team.run_stream(task=user_input)
        await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

Enter a message (type 'exit' to leave):  exit


# Two Agent Team

In [10]:
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=base_completion_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=base_completion_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [11]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=119), content="Leaves whisper in amber and gold,\nDancing to the ground underfoot cold.\nCrisp air holds secrets, sweet and shy,\nEach breath you take, a sigh of why.\n\nPumpkins carved, jack-o'-lantern faces lit,\nFires crackling with warm, cozy delight.\nApples ripe in the markets' embrace,\nColors bursting like nature's own race.\n\nChill breezes nudge us to remember,\nTime slips by as summer wavers and shivers.\nSo we gather close, share tales of home,\nIn the embrace of autumn where dreams roam.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=230), content="Your poem captures the essence of fall beautifully with its rich imagery and emotional resonance. Here are a few suggestions to enh

In [12]:
## Observe team in action
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=29, completion_tokens=127) content="Leaves whisper in soft decay,\nAs summer leaves bid farewell today.\nCrimson and gold, they dance and spin,\nIn the crisp autumn wind that begins.\n\nAcorns drift to earth with gentle grace,\nNature's lullaby, a peaceful race.\nPumpkins carved at Halloween's light,\nMajestic, glowing through the night.\n\nCozy sweaters for arms and knees,\nBy crackling fires where shadows please.\nApple pies and spicy cider sweet,\nFill the air with autumn's creamy treat.\n\nCooler breezes, crisper in our breath,\nUnder skies that are so vivid, they feel dear death." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=167, completion_tokens=73) content='Your poem captures the essence of fall beautifully! Here are a few suggestions to enhance it further:\n- Consider varying the rhyme sche

In [13]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves of gold and crimson hue,
Whisper tales as they drift away.
Crisp air nips at the cool blue,
As summer's warmth slowly decay.

Bare branches reach against the sky,
In shadows cast by clouds so gray.
Acorns fall from ancient pines,
Nature’s promise, come what may.

Frost hints of the chilling night,
Painting paths with silver light.
Fireplaces crackle and glow delightfully,
As stories fill the household’s hearth tonight.
---------- critic ----------
Your poem beautifully captures the essence of autumn. Here are a few suggestions to enhance it further:

1. Consider alternating between longer and shorter lines for varied rhythm and pace.
2. You might add more vivid imagery, such as describing leaves crunching underfoot or the scent of wood smoke.

Here's an adjusted version incorporating your feedback:
Leaves of gold and crimson hue,
Whisper tales as they gently drift away.
Crisp air n

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=101), content="Leaves of gold and crimson hue,\nWhisper tales as they drift away.\nCrisp air nips at the cool blue,\nAs summer's warmth slowly decay.\n\nBare branches reach against the sky,\nIn shadows cast by clouds so gray.\nAcorns fall from ancient pines,\nNature’s promise, come what may.\n\nFrost hints of the chilling night,\nPainting paths with silver light.\nFireplaces crackle and glow delightfully,\nAs stories fill the household’s hearth tonight.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=141, completion_tokens=176), content="Your poem beautifully captures the essence of autumn. Here are a few suggestions to enhance it further:\n\n1. Consider alternating between longer and shorter lines for varied rhyt

In [14]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.


# Human in the Loop

In [15]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
assistant = AssistantAgent("assistant", model_client=base_completion_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Whispers of waves caress the sand,
Blue depths hold secrets, vast and unplumbed,
Sunlight dances, kissing green crests goodbye,
Eternal blue heartbeats at life's tide.


Enter your response:  APPROVE


---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=41), content="Whispers of waves caress the sand,\nBlue depths hold secrets, vast and unplumbed,\nSunlight dances, kissing green crests goodbye,\nEternal blue heartbeats at life's tide.", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='a951553a-0adb-46e7-a21b-c279ff4db7a6', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

# Selector Group Chat (use model to select next speaker)

In [17]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [18]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100


In [19]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=base_completion_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Web search agent: Searches for information
        Data analyst: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="A web search agent.",
    tools=[search_web_tool],
    model_client=base_completion_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="A data analyst agent. Useful for performing calculations.",
    model_client=base_completion_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    """,
)

In [20]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=base_completion_client,
    termination_condition=termination,
)

In [21]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- WebSearchAgent ----------
[FunctionCall(id='call_u8n7iohq', arguments='{"query":"Miami Heat player with highest points in 2006-2007 season"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', call_id='call_u8n7iohq')]
---------- WebSearchAgent ----------
Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        
---------- DataAnalystAgent ----------
[FunctionCall(id='call_zgad0c4u',

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=240, completion_tokens=38), content=[FunctionCall(id='call_u8n7iohq', arguments='{"query":"Miami Heat player with highest points in 2006-2007 season"}', name='search_web_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WebSearchAgent', models_usage=None, content=[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', call_id='call_u8n7iohq')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='WebSearchAgent', mo

# Swarm

In [22]:
from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [23]:
def refund_flight(flight_id: str) -> str:
    """Refund a flight"""
    return f"Flight {flight_id} refunded"

In [24]:
travel_agent = AssistantAgent(
    "travel_agent",
    model_client=base_completion_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=base_completion_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [25]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)

In [25]:
task = "I need to refund my flight."


async def run_team_stream() -> None:
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]

    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        user_message = input("User: ")

        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]


await run_team_stream()

---------- user ----------
I need to refund my flight.
---------- travel_agent ----------
[FunctionCall(id='call_noeq68r7', arguments='{}', name='transfer_to_flights_refunder')]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', call_id='call_noeq68r7')]
---------- travel_agent ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- flights_refunder ----------
Could you please provide me with your flight reference number so I can process the refund?
---------- flights_refunder ----------

---------- flights_refunder ----------
[FunctionCall(id='call_zw89thwu', arguments='{}', name='transfer_to_user')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', call_id='call_zw89thwu')]
---------- flights_refunder ----------
Transferred to user, adopting the rol

User:  12345


---------- user ----------
12345
---------- flights_refunder ----------
[FunctionCall(id='call_u1byeg06', arguments='{"flight_id":"12345"}', name='refund_flight')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Flight 12345 refunded', call_id='call_u1byeg06')]
---------- flights_refunder ----------
Flight 12345 refunded
---------- flights_refunder ----------
[FunctionCall(id='call_p4apqgqy', arguments='{}', name='transfer_to_travel_agent')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to travel_agent, adopting the role of travel_agent immediately.', call_id='call_p4apqgqy')]
---------- flights_refunder ----------
Transferred to travel_agent, adopting the role of travel_agent immediately.
---------- travel_agent ----------
Your flight has been successfully refunded. Thank you for using our services!

If you have any other questions or need further assistance, feel free to ask.

TERMINATE


In [26]:
# Stock Research Swarm

In [27]:
async def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Get stock market data for a given symbol"""
    return {"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"}


async def get_news(query: str) -> List[Dict[str, str]]:
    """Get recent news articles about a company"""
    return [
        {
            "title": "Tesla Expands Cybertruck Production",
            "date": "2024-03-20",
            "summary": "Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.",
        },
        {
            "title": "Tesla FSD Beta Shows Promise",
            "date": "2024-03-19",
            "summary": "Latest Full Self-Driving beta demonstrates significant improvements in urban navigation and safety features.",
        },
        {
            "title": "Model Y Dominates Global EV Sales",
            "date": "2024-03-18",
            "summary": "Tesla's Model Y becomes best-selling electric vehicle worldwide, capturing significant market share.",
        },
    ]


In [28]:
planner = AssistantAgent(
    "planner",
    model_client=base_completion_client,
    handoffs=["financial_analyst", "news_analyst", "writer"],
    system_message="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
)

financial_analyst = AssistantAgent(
    "financial_analyst",
    model_client=base_completion_client,
    handoffs=["planner"],
    tools=[get_stock_data],
    system_message="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    Always handoff back to planner when analysis is complete.""",
)

news_analyst = AssistantAgent(
    "news_analyst",
    model_client=base_completion_client,
    handoffs=["planner"],
    tools=[get_news],
    system_message="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
)

writer = AssistantAgent(
    "writer",
    model_client=base_completion_client,
    handoffs=["planner"],
    system_message="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    Always handoff back to planner when writing is complete.""",
)

In [29]:
# Define termination condition
text_termination = TextMentionTermination("TERMINATE")
termination = text_termination

research_team = Swarm(
    participants=[planner, financial_analyst, news_analyst, writer], termination_condition=termination
)

task = "Conduct market research for TSLA stock"
await Console(research_team.run_stream(task=task))

---------- user ----------
Conduct market research for TSLA stock
---------- planner ----------
[FunctionCall(id='call_lpy00s1x', arguments='{}', name='transfer_to_financial_analyst')]
---------- planner ----------
[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', call_id='call_lpy00s1x')]
---------- planner ----------
Transferred to financial_analyst, adopting the role of financial_analyst immediately.
---------- financial_analyst ----------
[FunctionCall(id='call_20tywhwi', arguments='{"symbol":"TSLA"}', name='get_stock_data')]
---------- financial_analyst ----------
[FunctionExecutionResult(content="{'price': 180.25, 'volume': 1000000, 'pe_ratio': 65.4, 'market_cap': '700B'}", call_id='call_20tywhwi')]
---------- financial_analyst ----------
{'price': 180.25, 'volume': 1000000, 'pe_ratio': 65.4, 'market_cap': '700B'}
---------- financial_analyst ----------
Based on the provided data for Tesla (TSLA), here are so

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Conduct market research for TSLA stock', type='TextMessage'), ToolCallRequestEvent(source='planner', models_usage=RequestUsage(prompt_tokens=309, completion_tokens=21), content=[FunctionCall(id='call_lpy00s1x', arguments='{}', name='transfer_to_financial_analyst')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='planner', models_usage=None, content=[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', call_id='call_lpy00s1x')], type='ToolCallExecutionEvent'), HandoffMessage(source='planner', models_usage=None, target='financial_analyst', content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', context=[], type='HandoffMessage'), ToolCallRequestEvent(source='financial_analyst', models_usage=RequestUsage(prompt_tokens=246, completion_tokens=23), content=[FunctionCall(id='call_20tywhwi', argume

# MagneticONE

In [ ]:
# https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/magentic-one.html

In [ ]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console

model_client = OpenAIChatCompletionClient(
        model="qwen2.5",
        api_key="NotRequiredSinceWeAreLocal",
        base_url='http://host.docker.internal:11434/v1',
        model_capabilities={
            "json_output": True,
            "vision": False,
            "function_calling": True,
        },
        parallel_tool_calls=True
 )

async def main() -> None:

    assistant = AssistantAgent(
        "Assistant",
        model_client=base_completion_client,
    )
    team = MagenticOneGroupChat([assistant], model_client=model_client)
    await Console(team.run_stream(task="Provide a different proof for Fermat's Last Theorem"))


await main()

---------- user ----------
Provide a different proof for Fermat's Last Theorem
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

Provide a different proof for Fermat's Last Theorem


To answer this request we have assembled the following team:

Assistant: An agent that provides assistance with ability to use tools.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
   - The request is for a different proof of Fermat's Last Theorem.

2. FACTS TO LOOK UP
   - Where specifically: Detailed information on existing proofs and their methods can be found in academic papers, such as Wiles' Proof of Fermat's Last Theorem by Andrew Wiles published in the Annals of Mathematics.
   - Where specifically: History of mathematical concepts and methodologies relevant to the proof can be found in the references provided with historical texts or math journals.

3. FACTS TO DERIVE
   - Deriving a different approach would require unde